In [1]:
import ROOT
import numpy as np
import pandas as pd

Welcome to JupyROOT 6.18/04


In [2]:
def merge_files(file_ch1, file_ch2):
    df1 = pd.read_csv(file_ch1,'Channel 1',engine='python')
    df2 = pd.read_csv(file_ch2,'Channel 2',engine='python')
    res = pd.concat([df1,df2],axis=1)
    res.to_csv('merged_channels.dat',index=False)

In [3]:
file1 = './Labs/muon_271119/Run3/3052B-01_1_1_113001_2010_191127-175401.dat'
file2 = './Labs/muon_271119/Run3/3052B-01_2_2_113002_1410_191127-175401.dat'

merge_files(file1, file2)

In [2]:
dt = np.genfromtxt('merged_channels.dat',delimiter=',', usecols=[0,1], invalid_raise=False)

In [3]:
timestep_in_s = 10*dt[7,0]
timestep_in_ns = timestep_in_s*1E9
ayy = dt[9,0]
no_events = dt[10,0]
print(no_events)

1000.0


In [4]:
print("Time per event: {time} ns" .format(time = timestep_in_ns ))

Time per event: 200.0 ns


In [5]:
data_channels = dt[13:]
total_size = data_channels[:,0].size

In [6]:
points_per_event = int(round(timestep_in_s/ayy))
print('Number of point per events {points}'.format(points=points_per_event))
events = total_size//points_per_event
print('Number of events: {events}'.format(events=events))

Number of point per events 1000
Number of events: 1000


In [11]:
time_per_event_in_ns = timestep_in_ns/(points_per_event)
print(time_per_event_in_ns,timestep_in_ns)
timesteps_in_ns = np.arange(0,timestep_in_ns,time_per_event_in_ns)
print(timestep_in_ns)

0.2 200.0
200.0


In [24]:
f = ROOT.TFile('testing1.root', 'recreate')
tree = ROOT.TTree('channels', '')

In [25]:
ch1 = ROOT.std.vector('float')()
ch2 = ROOT.std.vector('float')()
timesteps = ROOT.std.vector('float')()

tree.Branch('ch1', ch1)
tree.Branch('ch2', ch2)
tree.Branch('timesteps', timesteps)

<ROOT.TBranchElement object ("timesteps") at 0x561edf782ce0>

In [26]:
# tree.SetEntries
for i in range(events):
    for data in data_channels[int(points_per_event*i):int(points_per_event*(i+1))]:
        ch1.push_back(data[0])
        ch2.push_back(data[1])    
    for i in range(points_per_event):  
        timesteps.push_back(timesteps_in_ns[i])
    tree.Fill()
    ch1.clear()
    ch1.shrink_to_fit()
    ch2.clear()
    ch2.shrink_to_fit()  
    timesteps.clear()
    timesteps.shrink_to_fit() 

In [27]:
f.Write()
f.Close()

In [14]:
N = events
x_ch1 = np.empty((N,points_per_event))
x_ch2 = np.empty((N,points_per_event))
for i in range(N):
    tempch1 = []
    tempch2 = []
    for data in data_channels[int(points_per_event*i):int(points_per_event*(i+1))]:
        tempch1.append(data[0])
        tempch2.append(data[1])        
    temp2ch1 = np.asarray(tempch1)
    temp2ch2 = np.asarray(tempch2)
    x_ch1[i] = np.transpose(temp2ch1)  
    x_ch2[i] = np.transpose(temp2ch2) 

In [15]:
# np.savetxt('ch1_values.csv', x_ch1, fmt="%1.4f", delimiter=",")
# np.savetxt('ch2_values.csv', x_ch2, fmt="%1.4f", delimiter=",")
np.savetxt('ch1_events_per_col.csv', np.transpose(x_ch1), fmt="%1.4f", delimiter=",")
np.savetxt('ch2_events_per_col.csv', np.transpose(x_ch2), fmt="%1.4f", delimiter=",")

In [ ]:
readHeaders = False

In [ ]:

x1 = ROOT.RDF.MakeCsvDataFrame('ch1_values.csv',readHeaders)

In [ ]:
x1.Snapshot('ch1','ch1RDF.root')

In [ ]:
x2 = ROOT.RDF.MakeCsvDataFrame('ch2_values.csv',readHeaders)
x2.Snapshot('ch2','ch2RDF.root')

In [ ]:


x1 = ROOT.RDF.MakeCsvDataFrame('ch1_events_per_col.csv',readHeaders)
x1.Snapshot('ch1','ch1RDF.root')